In [46]:
import pandas as pd
import numpy as np
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import spacy
import itertools as it
import os

In [ ]:
import nltk
from gensim import corpora
from collections import defaultdict
from pprint import pprint
import codecs

In [5]:
pd.set_option('display.max_colwidth', 100)

In [6]:
nlp = spacy.load('en') 
## python -m spacy download en

In [7]:
review = pd.read_csv("/Volumes/Transcend/MDS_Yelp/yelp_review.csv")

In [8]:
nv_reviews = review.sample(5000)
nv_reviews['parsed_text'] = None
nv_reviews.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,parsed_text
3904751,_v0ugLEPH8ZK2eRxZIpg0Q,WL_U3MBUfTkeHL9lYxoleg,djW8gh3JJ-__NCxx1YQaHg,5,2010-01-13,"Man, Biagio's just has the best donuts ever. Don't talk to me about Krispy Kreme (and those don'...",10,9,7,None
1957173,WSgR_FbeR7X5vBfjtXv4Pw,--iiD1TQzaXGxePZ1g5S2A,u73j2VQ3TGWdMO-AG7MABw,5,2016-06-01,I had never had chicken and waffles before coming to Lo-Lo's and now it is a staple for my Satur...,2,0,2,None
3222633,5D0yuk0eirxRbzhTrJDWMw,Vdt1BqSt278sXRtwcMeBwQ,81XqABPROlu8WIER-soVmA,5,2016-08-03,Not all JM's are the same but if you're in the area and are looking for a food option that may n...,0,0,0,None
2218724,TVQ2Rz17on3EgZmxLxuTuw,b7VnXEb51NBUv2lczurQjg,TgjB-wlAVzol6HoDpE8KYg,3,2017-04-01,Kinda underwhelming. Went for Saturday morning brunch and got the Shepherds biscuit. Sounded int...,0,0,0,None
3784613,_2Lkx8FUWvGOL6a7weoC9A,R_WixXtQughL_n5G4K97BQ,00Iemn8BmKrwtaFtt829kw,5,2017-01-29,David was awesome. We live in an older home we recently had re-landscaped. We chose Sprinkler Re...,0,0,0,None


In [9]:
type(nv_reviews['text'].values[0])

str

In [10]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [11]:
documents = nv_reviews['text'].tolist()

In [12]:
docs = []
for i in range(len(documents)):
    docs.append(nlp(documents[i]))

In [13]:
documents[0]

"Man, Biagio's just has the best donuts ever. Don't talk to me about Krispy Kreme (and those don't even SOUND good, by the way.... donuts are not supposed to be Krispy!!) or Dunkin' or Amy Joy. None of those others can even compare to the lovely donuts made fresh at Biagio's! \n\nUnfortunately, my family thinks it's funny to get a few dozen of these babies about once a week. And I always eat 2 of them because... well, I'm tubby and I just cannot resist them. They are the freshest, most delicious donuts you'll ever taste. \n\nAnd really, this is all coming from someone who isn't that big of a donut fan in the first place. \n\nTotally worth the drive across the river to BFEastlake to get your donut fix."

In [14]:
docs[0]

Man, Biagio's just has the best donuts ever. Don't talk to me about Krispy Kreme (and those don't even SOUND good, by the way.... donuts are not supposed to be Krispy!!) or Dunkin' or Amy Joy. None of those others can even compare to the lovely donuts made fresh at Biagio's! 

Unfortunately, my family thinks it's funny to get a few dozen of these babies about once a week. And I always eat 2 of them because... well, I'm tubby and I just cannot resist them. They are the freshest, most delicious donuts you'll ever taste. 

And really, this is all coming from someone who isn't that big of a donut fan in the first place. 

Totally worth the drive across the river to BFEastlake to get your donut fix.

In [15]:
token_lemma = [token.lemma_ for token in docs[1]]

In [16]:
unigram_review = [token.lemma_ for token in docs[0] if not punct_space(token)]

In [17]:
# remove any remaining stopwords
unigram_review = [term for term in unigram_review if term not in spacy.lang.en.STOP_WORDS]
# nlp.Defaults.stop_words works as well

In [18]:
unigram_review

['man',
 'biagio',
 "'s",
 'good',
 'donut',
 'talk',
 '-PRON-',
 'krispy',
 'kreme',
 'sound',
 'good',
 'way',
 'donut',
 'suppose',
 'krispy',
 'dunkin',
 'amy',
 'joy',
 'compare',
 'lovely',
 'donut',
 'fresh',
 'biagio',
 "'s",
 'unfortunately',
 '-PRON-',
 'family',
 'think',
 '-PRON-',
 'funny',
 'dozen',
 'baby',
 'week',
 '-PRON-',
 'eat',
 '2',
 '-PRON-',
 '-PRON-',
 'tubby',
 '-PRON-',
 'resist',
 '-PRON-',
 '-PRON-',
 'fresh',
 'delicious',
 'donut',
 '-PRON-',
 'taste',
 'come',
 'big',
 'donut',
 'fan',
 'place',
 'totally',
 'worth',
 'drive',
 'river',
 'bfeastlake',
 '-PRON-',
 'donut',
 'fix']

In [19]:
from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser

sentences = Text8Corpus(datapath('testcorpus.txt'))

In [20]:
sentences

In [21]:
datapath('testcorpus.txt')

'/anaconda3/lib/python3.6/site-packages/gensim/test/test_data/testcorpus.txt'

In [22]:
open('/anaconda3/lib/python3.6/site-packages/gensim/test/test_data/testcorpus.txt')

<_io.TextIOWrapper name='/anaconda3/lib/python3.6/site-packages/gensim/test/test_data/testcorpus.txt' mode='r' encoding='UTF-8'>

In [23]:
phrases = Phrases(sentences, min_count=1, threshold=1)
phrases
#https://stackoverflow.com/questions/35716121/how-to-extract-phrases-from-corpus-using-gensim

In [24]:
phrases[[u'trees', u'graph', u'minors']]

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['trees_graph', 'minors']

In [25]:
nv_reviews = nv_reviews.reset_index(drop=True)
nv_reviews.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,parsed_text
0,_v0ugLEPH8ZK2eRxZIpg0Q,WL_U3MBUfTkeHL9lYxoleg,djW8gh3JJ-__NCxx1YQaHg,5,2010-01-13,"Man, Biagio's just has the best donuts ever. Don't talk to me about Krispy Kreme (and those don'...",10,9,7,None
1,WSgR_FbeR7X5vBfjtXv4Pw,--iiD1TQzaXGxePZ1g5S2A,u73j2VQ3TGWdMO-AG7MABw,5,2016-06-01,I had never had chicken and waffles before coming to Lo-Lo's and now it is a staple for my Satur...,2,0,2,None
2,5D0yuk0eirxRbzhTrJDWMw,Vdt1BqSt278sXRtwcMeBwQ,81XqABPROlu8WIER-soVmA,5,2016-08-03,Not all JM's are the same but if you're in the area and are looking for a food option that may n...,0,0,0,None
3,TVQ2Rz17on3EgZmxLxuTuw,b7VnXEb51NBUv2lczurQjg,TgjB-wlAVzol6HoDpE8KYg,3,2017-04-01,Kinda underwhelming. Went for Saturday morning brunch and got the Shepherds biscuit. Sounded int...,0,0,0,None
4,_2Lkx8FUWvGOL6a7weoC9A,R_WixXtQughL_n5G4K97BQ,00Iemn8BmKrwtaFtt829kw,5,2017-01-29,David was awesome. We live in an older home we recently had re-landscaped. We chose Sprinkler Re...,0,0,0,None


In [26]:
review_txt_filepath = os.path.join('review_text_all.txt')

review_count = 0

# create & open a new file in write mode
with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:
    # if this review is not about a restaurant, skip to the next one
    # write the restaurant review as a line in the new file
    # escape newline characters in the original review text
    for i in range(len(nv_reviews['text'])):
        review_txt_file.write(nv_reviews['text'][i].replace('\n', '\\n') + '\n')
        review_count += 1

print(u'''Text from {:,} restaurant reviews written to the new txt file.'''.format(review_count))

Text from 5,000 restaurant reviews written to the new txt file.


In [27]:
nv_reviews['text']

0       Man, Biagio's just has the best donuts ever. Don't talk to me about Krispy Kreme (and those don'...
1       I had never had chicken and waffles before coming to Lo-Lo's and now it is a staple for my Satur...
2       Not all JM's are the same but if you're in the area and are looking for a food option that may n...
3       Kinda underwhelming. Went for Saturday morning brunch and got the Shepherds biscuit. Sounded int...
4       David was awesome. We live in an older home we recently had re-landscaped. We chose Sprinkler Re...
5       This is probably my favorite fried chicken place in the valley. Way better than the other big na...
6       The staff was great but not so sure I could say the second about the drink I ordered. Perhaps it...
7       Very poor service.  Made an appointment for 10 am and brought car in for emissions test and oil ...
8       just got two orders of baked mostacciolli.  One was obviously bad by the overwhelming smell when...
9       Super fun store to s

In [28]:
os.getcwd()

'/Users/rajeshpothamsetty/Dropbox/Rajesh_DB/IMP_Dropbox'

In [29]:
# filepaths# filepat 
unigram_sentences_filepath = os.path.join('unigram.txt')
bigram_sentences_filepath = os.path.join('bigrams.txt')
bigram_model_filepath = os.path.join('bigram_model_all')
trigram_sentences_filepath = os.path.join('trigrams.txt')
trigram_model_filepath = os.path.join('trigram_model_all')

In [30]:
#unigrams
with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        f.write(sentence + '\n')

unigram_sentences = LineSentence(unigram_sentences_filepath)



In [31]:
data1 = range(10)

# This creates a NEW list
data1[2:5]

# This creates an iterator that iterates over the EXISTING list
for x in it.islice(data1, 2, 5):
    print(x)

2
3
4


In [32]:
%%time
use_model = 'no'
# this is a bit time consuming - make the if statement True# this is 
# if you want to execute modeling yourself.
if use_model == 'no':
    bigram_model = Phrases(unigram_sentences)
    bigram_model.save(bigram_model_filepath)
else:    
    # load the finished model from disk
    bigram_model = Phrases.load(bigram_model_filepath)

CPU times: user 1.35 s, sys: 45.1 ms, total: 1.39 s
Wall time: 1.41 s


In [33]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if use_model == 'no':
    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for unigram_sentence in unigram_sentences:
            bigram_sentence = u' '.join(bigram_model[unigram_sentence]) 
            f.write(bigram_sentence + '\n')

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 2.84 s, sys: 18.4 ms, total: 2.86 s
Wall time: 2.87 s


In [34]:
bigram_sentences  = LineSentence(bigram_sentences_filepath)

In [35]:
%%time

use_model = 'no'
# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if use_model == 'no':
    trigram_model = Phrases(bigram_sentences)
    trigram_model.save(trigram_model_filepath)
else:    
    # load the finished model from disk
    trigram_model = Phrases.load(trigram_model_filepath)

CPU times: user 1.3 s, sys: 40.1 ms, total: 1.34 s
Wall time: 1.35 s


In [36]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if use_model == 'no':
    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:  
        for bigram_sentence in bigram_sentences:        
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])       
            f.write(trigram_sentence + '\n')

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 2.87 s, sys: 20.6 ms, total: 2.89 s
Wall time: 2.9 s


In [37]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [38]:
trigram_reviews_filepath  = os.path.join('trigram_transformed_reviews_all.txt')

In [39]:
%%time

use_model_trigram = 'no'
# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if use_model_trigram == 'no':
    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        for parsed_review in nlp.pipe(line_review(review_txt_filepath), batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review if term not in spacy.lang.en.STOP_WORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 2min 51s, sys: 41.3 s, total: 3min 32s
Wall time: 2min 12s


In [40]:
print(u'Original:' + u'\n')

for review in it.islice(line_review(review_txt_filepath), 10, 11):
    print(review)
    print(type(review))

print(u'----' + u'\n')
print(u'Transformed:' + u'\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 10, 11):
        temp = review
        print(review)
        print(type(review))

Original:

I heard on Facebook that its one of the best taco place in Vegas, so I finally tried it. They have friendly employees. 
I ordered 4 carne asada tacos to go with "everything" on it.  "Everything" on it... but it DOES NOT come with Guacamole. 
They make handmade tortilla... it was good! But the carne asada meat is bland and not juicy. 
The taco is pretty much tasteless and dry. I "think" (not 100% sure) its because they cooked a big batch whenever and just let it sit there, unlike other taco places. 
$2.50 per taco.... decent price for the size, but I can get with the same price, better seasoned, and freshly cooked carne asada meat with "Guacamole" at Pepe's or Tacos el Gordo and they cook the meat there when they get orders and not make big batch.

<class 'str'>
----

Transformed:

-PRON- hear facebook -PRON- good taco place in_vegas -PRON- finally try -PRON- -PRON- friendly employee -PRON- order 4 carne_asada taco -PRON- -PRON- -PRON- do_not come guacamole -PRON- handmade to

In [41]:
temp

"-PRON- hear facebook -PRON- good taco place in_vegas -PRON- finally try -PRON- -PRON- friendly employee -PRON- order 4 carne_asada taco -PRON- -PRON- -PRON- do_not come guacamole -PRON- handmade tortilla -PRON- good carne_asada meat bland juicy taco pretty_much tasteless dry -PRON- think 100 sure -PRON- -PRON- cook big batch let -PRON- sit unlike_other taco place $_2.50 taco decent price size -PRON- price better season freshly cook carne_asada meat guacamole pepe 's tacos el gordo -PRON- cook meat -PRON- order big batch\n"

In [42]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100)

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [43]:
# from linkedin course
# not a great combination with tri-gram
# should try independently
x_temp = temp.replace('_', " ")
clean_text(temp) 
# remove 'pron'
# add spell Checker
# 

['pron',
 'hear',
 'facebook',
 'pron',
 'good',
 'taco',
 'place',
 'invega',
 'pron',
 'final',
 'tri',
 'pron',
 'pron',
 'friendli',
 'employe',
 'pron',
 'order',
 '4',
 'carneasada',
 'taco',
 'pron',
 'pron',
 'pron',
 'donot',
 'come',
 'guacamol',
 'pron',
 'handmad',
 'tortilla',
 'pron',
 'good',
 'carneasada',
 'meat',
 'bland',
 'juici',
 'taco',
 'prettymuch',
 'tasteless',
 'dri',
 'pron',
 'think',
 '100',
 'sure',
 'pron',
 'pron',
 'cook',
 'big',
 'batch',
 'let',
 'pron',
 'sit',
 'unlikeoth',
 'taco',
 'place',
 '250',
 'taco',
 'decent',
 'price',
 'size',
 'pron',
 'price',
 'better',
 'season',
 'freshli',
 'cook',
 'carneasada',
 'meat',
 'guacamol',
 'pepe',
 'taco',
 'el',
 'gordo',
 'pron',
 'cook',
 'meat',
 'pron',
 'order',
 'big',
 'batch',
 '']

In [44]:
clean_text(x_temp)

['pron',
 'hear',
 'facebook',
 'pron',
 'good',
 'taco',
 'place',
 'vega',
 'pron',
 'final',
 'tri',
 'pron',
 'pron',
 'friendli',
 'employe',
 'pron',
 'order',
 '4',
 'carn',
 'asada',
 'taco',
 'pron',
 'pron',
 'pron',
 'come',
 'guacamol',
 'pron',
 'handmad',
 'tortilla',
 'pron',
 'good',
 'carn',
 'asada',
 'meat',
 'bland',
 'juici',
 'taco',
 'pretti',
 'much',
 'tasteless',
 'dri',
 'pron',
 'think',
 '100',
 'sure',
 'pron',
 'pron',
 'cook',
 'big',
 'batch',
 'let',
 'pron',
 'sit',
 'unlik',
 'taco',
 'place',
 '250',
 'taco',
 'decent',
 'price',
 'size',
 'pron',
 'price',
 'better',
 'season',
 'freshli',
 'cook',
 'carn',
 'asada',
 'meat',
 'guacamol',
 'pepe',
 'taco',
 'el',
 'gordo',
 'pron',
 'cook',
 'meat',
 'pron',
 'order',
 'big',
 'batch',
 '']

In [45]:
acac = open('trigram_transformed_reviews_all.txt', "r")
lines = list(acac)
lines_ = pd.Series(lines)

In [46]:
lines_

0       man biagio 's good donut do_not talk -PRON- krispy kreme do_not sound good way donut suppose kri...
1       -PRON- chicken waffle come lo lo 's -PRON- staple -PRON- saturday_morning hangover routine great...
2       jm 's -PRON- area look food option drive_through -PRON- away_from deep fryer -PRON- find 80 -PRO...
3       kinda underwhelm saturday_morning brunch shepherds biscuit sound interesting end_up flavorless d...
4       david awesome -PRON- live an_old home -PRON- recently landscape -PRON- choose sprinkler repair b...
5                    probably -PRON- favorite fry_chicken place valley way better_than big national chain\n
6       staff great sure -PRON- second drink -PRON- order -PRON- strong iced americano -PRON- order extr...
7       poor_service make_an_appointment 10 bring car emission test oil_change check_out car an_hour -PR...
8       order baked mostacciolli obviously bad overwhelming smell -PRON- open lid -PRON- offer send -PRO...
9       super fun store shop

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(lines_)
print(X_tfidf.shape)
print(tfidf_vect.get_feature_names())

(5008, 16791)
['', '0', '00', '01', '02', '0323', '032512', '04052013', '041111', '050', '05042012', '06', '075', '0900', '1', '10', '100', '1000', '10000', '100000', '1000th', '10067', '100am', '100degre', '100night', '100pm', '100yr', '101', '1010', '1015', '101710', '10211', '1022', '1025', '1027', '1030', '1030pm', '10365', '1045', '1045am', '1045pm', '105', '1050', '105f', '106', '107', '107000', '1079', '108a113', '108f', '1095', '109790', '1099', '10am', '10am11pm', '10am2pm', '10am4pm', '10class', '10extram', '10k', '10min', '10minut', '10off', '10piec', '10pm', '10pp', '10slice', '10star', '10th', '10x', '10year', '11', '110', '1100', '1100am', '1106', '1110pm', '1113', '1115pm', '112', '1120am', '1120pm', '1129', '1130', '113010', '1130am', '1130ish', '1130pm', '1133', '1135pm', '1142am', '1143pm', '1145', '115', '1150', '11511', '1162015', '11620151', '117b', '117th', '11894', '1195', '1199', '11a', '11am', '11great', '11month', '11pm', '11th', '12', '120', '1200', '1210', '

In [48]:
data_sample = lines_[0:20]

tfidf_vect_sample = TfidfVectorizer(analyzer=clean_text)
X_tfidf_sample = tfidf_vect_sample.fit_transform(data_sample)
print(X_tfidf_sample.shape)
print(tfidf_vect_sample.get_feature_names())

X_tfidf_df = pd.DataFrame(X_tfidf_sample.toarray())
X_tfidf_df.columns = tfidf_vect_sample.get_feature_names()
X_tfidf_df

(20, 605)
['', '10', '100', '1145', '18', '1850', '2', '2017', '22', '250', '3rd', '4', '5', '500', '80', 'aaa', 'abel', 'ableto', 'acquaint', 'actual', 'address', 'adult', 'advis', 'afew', 'age', 'agenc', 'aioli', 'alfredo', 'amaz', 'americano', 'ami', 'angi', 'anhour', 'anold', 'anyway', 'approv', 'area', 'arugula', 'ask', 'aswel', 'attach', 'attend', 'attitud', 'avocado', 'awayfrom', 'awesom', 'babi', 'bacon', 'bad', 'bake', 'base', 'batch', 'beauti', 'beef', 'best', 'besti', 'better', 'betterthan', 'bfeastlak', 'biagio', 'big', 'biscuit', 'bitter', 'bland', 'blow', 'booti', 'boozi', 'brake', 'brandnew', 'breakfast', 'brilliant', 'bring', 'brunch', 'bumper', 'bun', 'burger', 'busi', 'cannot', 'car', 'careu', 'carneasada', 'carpetclean', 'case', 'cashier', 'cc', 'certif', 'chain', 'charg', 'charger', 'check', 'checkout', 'cheddarchees', 'cheesecak', 'chef', 'chemic', 'chicken', 'chip', 'chive', 'chocol', 'choos', 'clear', 'client', 'cloth', 'coffe', 'color', 'come', 'comeout', 'comfo

,,10,100,1145,18,1850,2,2017,22,250,...,wing,word,work,worri,worth,wouldhighlyrecommend,wouldrecommend,yam,yelper,yelpreview
0,0.031156,0.000000,0.000000,0.000000,0.000000,0.000000,0.091784,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,0.050893,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
2,0.042919,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,0.040069,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,0.042757,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.104116,0.00000,0.000000,0.143294,0.000000,0.000000,0.00000,0.143294
5,0.104047,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
6,0.058548,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
7,0.012125,0.040634,0.000000,0.040634,0.000000,0.000000,0.035718,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.040634,0.000000,0.000000,0.000000,0.00000,0.000000
8,0.076370,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
9,0.052828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [49]:
from collections import defaultdict
dict_series = [0]*len(lines_)
i = 0
for rev in lines_:
    fq= defaultdict(int)
    for w in rev.split():
        fq[w] += 1
    dict_series[i] = fq
    i += 1

In [50]:
fq

defaultdict(int,
            {'-PRON-': 2,
             'beer': 1,
             'bomb': 1,
             'bonus': 1,
             'chandler': 1,
             'definitely': 1,
             'delicious': 1,
             'dinner': 1,
             'great': 1,
             'italian_food': 1,
             'key': 1,
             'list': 1,
             'low': 1,
             'muffins': 1,
             'pepperoni': 1,
             'pitcher': 1,
             'place': 1,
             'reasonably_price': 1,
             'serve': 1,
             'vibe': 1})

In [51]:
# https://programminghistorian.org/en/lessons/counting-frequencies
dict_series[0:10]

[defaultdict(int,
             {"'s": 2,
              '-PRON-': 11,
              '2': 1,
              'a_few': 1,
              'amy': 1,
              'baby': 1,
              'bfeastlake': 1,
              'biagio': 2,
              'big': 1,
              'can_not': 1,
              'come': 1,
              'compare': 1,
              'delicious': 1,
              'do_not': 2,
              'donut': 6,
              'dozen': 1,
              'drive': 1,
              'dunkin': 1,
              'eat': 1,
              'family': 1,
              'fan': 1,
              'fix': 1,
              'fresh': 2,
              'funny': 1,
              'good': 2,
              'joy': 1,
              'kreme': 1,
              'krispy': 2,
              'lovely': 1,
              'man': 1,
              'none_of': 1,
              'place': 1,
              'resist': 1,
              'river': 1,
              'someone_who': 1,
              'sound': 1,
              'suppose': 1,
            

In [52]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(zip(wordlist,wordfreq))

review_series = [wordListToFreqDict(wordstring.split()) for wordstring in lines_]

In [53]:
review_series[0:5]

[{"'s": 2,
  '-PRON-': 11,
  '2': 1,
  'a_few': 1,
  'amy': 1,
  'baby': 1,
  'bfeastlake': 1,
  'biagio': 2,
  'big': 1,
  'can_not': 1,
  'come': 1,
  'compare': 1,
  'delicious': 1,
  'do_not': 2,
  'donut': 6,
  'dozen': 1,
  'drive': 1,
  'dunkin': 1,
  'eat': 1,
  'family': 1,
  'fan': 1,
  'fix': 1,
  'fresh': 2,
  'funny': 1,
  'good': 2,
  'joy': 1,
  'kreme': 1,
  'krispy': 2,
  'lovely': 1,
  'man': 1,
  'none_of': 1,
  'place': 1,
  'resist': 1,
  'river': 1,
  'someone_who': 1,
  'sound': 1,
  'suppose': 1,
  'talk': 1,
  'taste': 1,
  'think': 1,
  'totally_worth': 1,
  'tubby': 1,
  'unfortunately': 1,
  'way': 1,
  'week': 1},
 {"'s": 1,
  '-PRON-': 7,
  'busy': 1,
  'can_not': 1,
  'chicken': 2,
  'come': 2,
  'comfort_food': 1,
  'delicious': 1,
  'favorite': 1,
  'food': 1,
  'go_wrong': 1,
  'great': 1,
  'hangover': 1,
  'lo': 2,
  'menu': 1,
  'ok': 1,
  'people': 1,
  'place': 1,
  'routine': 1,
  'saturday_morning': 1,
  'service': 1,
  'southern': 1,
  'staple'

In [54]:
# find a way to remove words which are not specific to restaurants
# idea: pick a business category which has very less similarity to 
        # restaurants and remove those words from our word lists

In [55]:
# Aggregating businesses by combined reviews will help in identifying similar restaurants.
    # can use this for recommending new restaurants
    # can also use this in identifying similar users (i.e similar restaurants liked by diff users)
# Aggregating user's by combining reviews will help in identifying similar restaurants.
    # we will get some sort of score
# probably could use that for users



In [56]:
### Spell checking
### english words has a max of two repeated letters consecutively
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

print(reduce_lengthening( "finallllllly" ))

finally


In [57]:
#MODULE = '/Users/rajeshpothamsetty/Downloads/pattern-master/pattern'
#import sys
#if MODULE not in sys.path: sys.path.append(MODULE)
#import pattern

from pattern3.en import spelling, suggest

word = "amazzziiing"
word_wlf = reduce_lengthening(word) #calling function defined above
print(word_wlf) #word lengthening isn't being able to fix it completely

correct_word = suggest(word_wlf) 
print(correct_word)

IndentationError: expected an indented block (tree.py, line 37)

In [58]:
from textblob import TextBlob
from textblob import Word

temp_textblob = TextBlob(x_temp)
#print(temp_textblob.correct())

In [59]:
temp_correct = temp_textblob.correct()
temp_correct

TextBlob("-PRON- hear facebook -PRON- good tact place in began -PRON- finally try -PRON- -PRON- friendly employee -PRON- order 4 care aside tact -PRON- -PRON- -PRON- do not come guacamole -PRON- handmade gorilla -PRON- good care aside meat bland juice tact pretty much tactless dry -PRON- think 100 sure -PRON- -PRON- cook big batch let -PRON- sit unlike other tact place $ 2.50 tact decent price size -PRON- price better season freshly cook care aside meat guacamole pipe 's faces el gordon -PRON- cook meat -PRON- order big batch
")

In [60]:
clean_text(temp_correct) #removes 'e' from arrive; why?

['pron',
 'hear',
 'facebook',
 'pron',
 'good',
 'tact',
 'place',
 'began',
 'pron',
 'final',
 'tri',
 'pron',
 'pron',
 'friendli',
 'employe',
 'pron',
 'order',
 '4',
 'care',
 'asid',
 'tact',
 'pron',
 'pron',
 'pron',
 'come',
 'guacamol',
 'pron',
 'handmad',
 'gorilla',
 'pron',
 'good',
 'care',
 'asid',
 'meat',
 'bland',
 'juic',
 'tact',
 'pretti',
 'much',
 'tactless',
 'dri',
 'pron',
 'think',
 '100',
 'sure',
 'pron',
 'pron',
 'cook',
 'big',
 'batch',
 'let',
 'pron',
 'sit',
 'unlik',
 'tact',
 'place',
 '250',
 'tact',
 'decent',
 'price',
 'size',
 'pron',
 'price',
 'better',
 'season',
 'freshli',
 'cook',
 'care',
 'asid',
 'meat',
 'guacamol',
 'pipe',
 'face',
 'el',
 'gordon',
 'pron',
 'cook',
 'meat',
 'pron',
 'order',
 'big',
 'batch',
 '']

In [61]:
TextBlob('reserv').correct()

TextBlob("reserve")

In [62]:
TextBlob('arriv').correct()

TextBlob("arrive")

In [63]:
## Pyspark Basics
## https://www.dezyre.com/apache-spark-tutorial/pyspark-tutorial

In [15]:
from operator import add
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.ml.feature import IDF
from pyspark.ml import Pipeline, PipelineModel

from pyspark.sql.functions import *
from pyspark.sql.types import *

import folium
import html

In [16]:
data_path = '/Volumes/Transcend/dataset/'
model_path = '/Volumes/Transcend/MDS_Yelp/model/'
outout_path = '/Volumes/Transcend/MDS_Yelp/output/'

In [17]:
business_df = spark.read.parquet(data_path + 'business-small.parquet')
business_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- business_name: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [18]:
user_df = spark.read.parquet(data_path + 'user-small.parquet')
user_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- yelping_since: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- fans: long (nullable = true)
 |-- average_stars: double (nullable = true)



In [19]:
review_df = spark.read.parquet(data_path + 'review-small.parquet')
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)



In [20]:
# create SQL view for later queries
review_df.createOrReplaceTempView("reviews")

# create review text dataframe
reviews_text = spark.sql("SELECT user_id, review_text FROM reviews")
reviews_text.show(3)

+--------------------+--------------------+
|             user_id|         review_text|
+--------------------+--------------------+
|u0LXt3Uea_GidxRW1...|Who would have gu...|
|u0LXt3Uea_GidxRW1...|Not bad!! Love th...|
|u0LXt3Uea_GidxRW1...|This is currently...|
+--------------------+--------------------+
only showing top 3 rows



In [21]:
# concatenate all reviews per restuarant

reviews_text_rdd = reviews_text.rdd
reviews_by_user_rdd = reviews_text_rdd.map(tuple).reduceByKey(add)  
reviews_by_user_df = spark.createDataFrame(reviews_by_user_rdd)
reviews_by_user_df = reviews_by_user_df \
                            .withColumnRenamed('_1', 'user_id') \
                            .withColumnRenamed('_2', 'text')
reviews_by_user_df.count()

73041

In [22]:
reviews_by_user_df.head()

Row(user_id='nOTl4aPC4tKHK35T3bNauQ', text='I give up on this place! I have gotten burritos and other things from this place on several occasions and it\'s always been sub-par. Bland food, overcooked, sometimes burnt (nachos). As a result, I haven\'t bothered going or ordering from there in well over a year. Well, today I had a craving for Mexican and since Toronto has a huge lack of Mexican restaurant choices and because I\'m busy working at home, my husband and myself decided to order from BarBurrito\'s delivery. I should also mention that if you go to justeat.ca BarBurrito charges $7 delivery. Orderit.ca has no charge. Don\'t know if that\'s the ordering website or the restaurant, but it\'s a tip for anyone who decides to order from this place.  So I went ahead and placed the order...only to have it be "rejected" by BarBurrito, because they are busy right now. This restaurant is listed on two different food ordering sites, but they won\'t deliver if they don\'t feel inclined?? SERIO

In [23]:
reviews_by_user_df.show(3)

+--------------------+--------------------+
|             user_id|                text|
+--------------------+--------------------+
|nOTl4aPC4tKHK35T3...|I give up on this...|
|Aj2IZibnWlSD1wWdq...|Love this place m...|
|6KM2pWFcsHYf4Nr9c...|Phenomenal brunch...|
+--------------------+--------------------+
only showing top 3 rows



In [24]:
## Example of using Word2vec
from pyspark.ml.feature import Word2Vec

# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)

result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [Hi, I, heard, about, Spark] => 
Vector: [-0.011087720049545169,-0.04713543206453324,-0.030747681856155396]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.02089989078896386,0.025788720164980204,-0.029980793063129695]

Text: [Logistic, regression, models, are, neat] => 
Vector: [0.03326573260128498,0.010403224825859071,-0.04281422272324562]



In [25]:
## https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.Word2Vec

## The minimum number of times a token must appear to be included in the word2vec model's vocabulary"

Tuning of vectorSize and minCount required in word2vec model training.

In [76]:
%%time

# create text processing pipeline -- this a lengthy resource-intensive process
# Build the pipeline 
regexTokenizer = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'text', outputCol = 'token')
stopWordsRemover = StopWordsRemover(inputCol = 'token', outputCol = 'nostopwrd')
countVectorizer = CountVectorizer(inputCol="nostopwrd", outputCol="rawFeature")
iDF = IDF(inputCol="rawFeature", outputCol="idf_vec")
word2Vec = Word2Vec(vectorSize = 100, minCount = 5, inputCol = 'nostopwrd', outputCol = 'word_vec', seed=123)
vectorAssembler = VectorAssembler(inputCols=['idf_vec', 'word_vec'], outputCol='comb_vec')
pipeline = Pipeline(stages=[regexTokenizer, stopWordsRemover, countVectorizer, iDF, word2Vec, vectorAssembler])

# fit the model
pipeline_mdl = pipeline.fit(reviews_by_user_df)

#save the pipeline model
pipeline_mdl.write().overwrite().save(model_path + 'pipe_txt')

In [26]:
# load the text transformation pipeline trained model
pipeline_mdl = PipelineModel.load(model_path + 'pipe_txt')

In [27]:
# transform the review data
reviews_by_user_trf_df = pipeline_mdl.transform(reviews_by_user_df)

In [28]:
# show the transformed review data
reviews_by_user_trf_df.select( 'text', 'nostopwrd', 'idf_vec', 'word_vec', 'comb_vec').show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|           nostopwrd|             idf_vec|            word_vec|            comb_vec|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|I give up on this...|[give, place, got...|(128365,[0,1,2,3,...|[-0.0374168294159...|(128465,[0,1,2,3,...|
|Love this place m...|[love, place, mak...|(128365,[0,1,2,3,...|[-0.0216792637887...|(128465,[0,1,2,3,...|
|Phenomenal brunch...|[phenomenal, brun...|(128365,[0,1,2,3,...|[-0.0473526970104...|(128465,[0,1,2,3,...|
|A born and bred T...|[born, bred, toro...|(128365,[0,1,2,3,...|[-0.0298230579082...|(128465,[0,1,2,3,...|
|We really enjoyed...|[really, enjoyed,...|(128365,[0,1,2,3,...|[-0.0115986177697...|(128465,[0,1,2,3,...|
|I was pretty unim...|[pretty, unimpres...|(128365,[0,1,2,3,...|[-0.0381316097274...|(128465,[0,1,2,3,...|
|I have never done...|[never, done, o

In [35]:
reviews_by_user_trf_df.select( 'nostopwrd').show(10)

+--------------------+
|           nostopwrd|
+--------------------+
|[give, place, got...|
|[love, place, mak...|
|[phenomenal, brun...|
|[born, bred, toro...|
|[really, enjoyed,...|
|[pretty, unimpres...|
|[never, done, one...|
|[seven, lives, se...|
|[best, indian, fo...|
|[good, ramen, bet...|
+--------------------+
only showing top 10 rows



In [36]:
reviews_by_user_trf_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nostopwrd: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rawFeature: vector (nullable = true)
 |-- idf_vec: vector (nullable = true)
 |-- word_vec: vector (nullable = true)
 |-- comb_vec: vector (nullable = true)



In [37]:
type(iDF)

pyspark.ml.feature.IDF

In [38]:
def CosineSim(vec1, vec2): 
    return np.dot(vec1, vec2) / np.sqrt(np.dot(vec1, vec1)) / np.sqrt(np.dot(vec2, vec2))

In [39]:
all_user_vecs = reviews_by_user_trf_df.select('user_id', 'word_vec').rdd.map(lambda x: (x[0], x[1])).collect()

In [40]:
type(all_user_vecs)

list

In [58]:
CosineSim(all_user_vecs[0][1], all_user_vecs[1][1])

0.8652645262685584

In [41]:
all_user_vecs[0]
# user_id = 'nOTl4aPC4tKHK35T3bNauQ'
# DenseVector() - vector respresentation of all the reviews of User

('nOTl4aPC4tKHK35T3bNauQ',
 DenseVector([-0.0374, -0.0827, -0.0607, 0.0706, 0.0262, -0.0358, -0.0004, -0.009, -0.0103, -0.0207, -0.0211, -0.0439, -0.0347, 0.0174, -0.0089, -0.0437, -0.0669, -0.0098, -0.0301, -0.0068, -0.0761, -0.0492, 0.0009, 0.0052, 0.0626, -0.0469, -0.0064, -0.0302, -0.0117, 0.0336, 0.0578, 0.0054, 0.0243, -0.0307, -0.0272, -0.0705, 0.0969, 0.0265, -0.0284, -0.0408, -0.0362, -0.0576, -0.0311, 0.0778, 0.0081, -0.0428, -0.048, 0.0279, -0.0118, 0.014, 0.0095, -0.0122, -0.0123, -0.0328, -0.0584, 0.0126, 0.0096, 0.0456, 0.0413, 0.0209, -0.0138, -0.0004, 0.0237, -0.0566, -0.0347, 0.0297, 0.0447, 0.0419, -0.0252, 0.0758, 0.0014, -0.0045, 0.0094, 0.0375, 0.0088, -0.0188, -0.0655, 0.0098, -0.0166, 0.0423, 0.001, -0.0137, -0.0024, -0.0112, -0.0131, -0.0028, 0.0135, 0.0187, -0.0107, 0.0014, -0.0016, -0.0062, -0.0262, 0.0269, -0.0136, -0.0092, -0.0441, 0.0108, -0.0502, 0.025]))

In [42]:
all_user_vecs[30]

('QBac9-Ii6jR-yLsQ5MVTHg',
 DenseVector([-0.0437, -0.0333, -0.1018, 0.1128, 0.019, -0.04, -0.0025, -0.0092, 0.008, -0.0077, -0.009, -0.0322, 0.0118, -0.0193, 0.0048, -0.0507, -0.0476, 0.0128, -0.0378, 0.0069, -0.0707, -0.0564, 0.0284, 0.0181, 0.0311, -0.0848, -0.0189, -0.0515, -0.013, 0.0437, 0.0362, 0.0224, 0.0427, -0.0589, -0.0502, -0.0365, 0.0593, 0.0084, 0.014, -0.0671, -0.0321, -0.0356, -0.029, 0.0449, 0.0081, -0.0594, -0.0594, 0.0502, 0.0344, -0.0147, -0.0015, -0.0193, -0.0214, -0.0669, -0.0962, 0.0036, 0.0078, 0.015, 0.0491, 0.0059, -0.0501, 0.007, -0.0012, -0.055, -0.0339, 0.0223, 0.0317, 0.049, -0.0275, 0.0699, 0.0057, -0.017, -0.0013, 0.07, 0.0291, -0.0336, -0.0062, 0.0489, -0.0555, 0.0313, 0.0281, -0.0541, 0.0052, 0.0096, -0.0326, -0.0045, 0.0374, 0.0177, 0.0256, 0.0323, 0.0159, -0.053, -0.0328, 0.0331, 0.015, -0.0199, -0.0616, -0.0037, -0.038, 0.02]))

In [ ]:
# To do consider only if the USer had minm. number of reviews
# Tuning VectorSize and minCount
# maybe removing non-restaurant related words...

In [68]:
def getSimilarUsers(b_ids, sim_user_limit=10):
    
    schema = StructType([   
                            StructField("user_id", StringType(), True)
                            ,StructField("score", IntegerType(), True)
                            ,StructField("input_user_id", StringType(), True)
                        ])
    
    similar_user_df = spark.createDataFrame([], schema)
    
    for b_id in b_ids:
        
        input_vec = [(r[1]) for r in all_user_vecs if r[0] == b_id][0]
        
        similar_user_rdd = sc.parallelize((i[0], float(CosineSim(input_vec, i[1]))) for i in all_user_vecs)
        
        similar_user_df = spark.createDataFrame(similar_user_rdd) \
            .withColumnRenamed('_1', 'user_id') \
            .withColumnRenamed('_2', 'score') \
            .orderBy("score", ascending = False)
            
        similar_user_df = similar_user_df.dropna()    
        similar_user_df = similar_user_df.filter(col("user_id") != b_id).limit(sim_user_limit)
        similar_user_df = similar_user_df.withColumn('input_user_id', lit(b_id))
        
        similar_user_df = similar_user_df \
                                    .union(similar_user_df)
        
    
    return similar_user_df

In [69]:
def getUserDetails(in_bus):
    
    a = in_bus.alias("a")
    b = user_df.alias("b")
    
    return a.join(b, col("a.user_id") == col("b.user_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.user_id'), col('b.user_name'),col('b.review_count')])

In [70]:
# test with two restaurants

uids = ['nOTl4aPC4tKHK35T3bNauQ', 'QBac9-Ii6jR-yLsQ5MVTHg']

print('\ninput user details:')
user_df.select('user_id','user_name', 'review_count') \
    .filter(user_df.user_id.isin(uids) == True).show(truncate=False)
    
# get top 10 similar business
sim_users = getUserDetails(getSimilarUsers(uids))

print('Top 10 similar Users for each input restaurant are:"')
sim_users.select('input_user_id', 'a.user_id', 'user_name', 'score','review_count').toPandas()


input user details:
+----------------------+---------+------------+
|user_id               |user_name|review_count|
+----------------------+---------+------------+
|nOTl4aPC4tKHK35T3bNauQ|Katherine|148         |
|QBac9-Ii6jR-yLsQ5MVTHg|Alex     |13          |
+----------------------+---------+------------+



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


Top 10 similar Users for each input restaurant are:"


,input_user_id,user_id,user_name,score,review_count
0,QBac9-Ii6jR-yLsQ5MVTHg,MpN81tQOL86GaFse-_tTRQ,Amy,0.913263,46
1,QBac9-Ii6jR-yLsQ5MVTHg,MpN81tQOL86GaFse-_tTRQ,Amy,0.913263,46
2,QBac9-Ii6jR-yLsQ5MVTHg,bPUpO-bP6BmAGvSwPyDsng,Michael,0.905848,142
3,QBac9-Ii6jR-yLsQ5MVTHg,bPUpO-bP6BmAGvSwPyDsng,Michael,0.905848,142
4,QBac9-Ii6jR-yLsQ5MVTHg,zkKv10FEw-iWJBHvK0mGfA,Paul,0.903733,55
5,QBac9-Ii6jR-yLsQ5MVTHg,zkKv10FEw-iWJBHvK0mGfA,Paul,0.903733,55
6,QBac9-Ii6jR-yLsQ5MVTHg,eV5usRjY2cDqNKVv8wXroA,Sam,0.905077,105
7,QBac9-Ii6jR-yLsQ5MVTHg,eV5usRjY2cDqNKVv8wXroA,Sam,0.905077,105
8,QBac9-Ii6jR-yLsQ5MVTHg,xcBcIMVWEx1p7V8ACaeHpQ,Rick,0.918668,83
9,QBac9-Ii6jR-yLsQ5MVTHg,xcBcIMVWEx1p7V8ACaeHpQ,Rick,0.918668,83


In [54]:
getSimilarUsers(uids)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


DataFrame[user_id: string, score: double, input_user_id: string]

In [71]:
getSimilarUsers(uids).head(20)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


[Row(user_id='xcBcIMVWEx1p7V8ACaeHpQ', score=0.9186675960618101, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'),
 Row(user_id='kw-YtOKPXrRB2a9wRZlmzQ', score=0.9154528761077289, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'),
 Row(user_id='MpN81tQOL86GaFse-_tTRQ', score=0.913263481576842, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'),
 Row(user_id='cNhHuEQMIpLH_qc9qGz67A', score=0.9107741561512877, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'),
 Row(user_id='g5W7s0n19gvT1Ujy_ITJog', score=0.9078462909984318, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'),
 Row(user_id='J5Eb7LhJaOa20k0ppcOCOg', score=0.9076639247312002, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'),
 Row(user_id='_IR48ok0ZkPMWJ2PlRCk0A', score=0.9070984934892682, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'),
 Row(user_id='bPUpO-bP6BmAGvSwPyDsng', score=0.9058483330903827, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'),
 Row(user_id='eV5usRjY2cDqNKVv8wXroA', score=0.9050774344012574, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'),
 Row(user_id='zkKv10FEw-iWJBHvK0mGfA', score=0.

In [59]:
schema = StructType([   
                        StructField("user_id", StringType(), True)
                        ,StructField("score", IntegerType(), True)
                        ,StructField("input_user_id", StringType(), True)
                    ])

similar_user_df = spark.createDataFrame([], schema)

In [ ]:
sim_user_limit = 10

for b_id in uids:
    print(b_id)
    input_vec = [(r[1]) for r in all_user_vecs if r[0] == b_id][0]
    #input_vec = reviews_by_business_trf_df.select('word_vec')\
                #.filter(reviews_by_business_trf_df['business_id'] == b_id)\
                #.collect()[0][0]

    similar_user_rdd = sc.parallelize((i[0], float(CosineSim(input_vec, i[1]))) for i in all_user_vecs)

    similar_user_df = spark.createDataFrame(similar_user_rdd) \
        .withColumnRenamed('_1', 'user_id') \
        .withColumnRenamed('_2', 'score') \
        .orderBy("score", ascending = False)
    similar_user_df = similar_user_df.dropna()
    #.where(isnan(col("a")))
    similar_user_df = similar_user_df.filter(col("user_id") != b_id).limit(sim_user_limit)
    similar_user_df = similar_user_df.withColumn('input_user_id', lit(b_id))

    similar_user_df = similar_user_df \
                                .union(similar_user_df)

    print("-------------------------------------")
    print(similar_user_df.take(10))
    print("-------------------------------------")

nOTl4aPC4tKHK35T3bNauQ


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


-------------------------------------
[Row(user_id='uO1w3qNo21c1bVHHFTYW0w', score=0.9722554149044773, input_user_id='nOTl4aPC4tKHK35T3bNauQ'), Row(user_id='myrcQ3h2G04Gv-ANG_oqrg', score=0.9713812297786633, input_user_id='nOTl4aPC4tKHK35T3bNauQ'), Row(user_id='VVm-TFCpi9M1-k8ED0l1eA', score=0.9710125785824025, input_user_id='nOTl4aPC4tKHK35T3bNauQ'), Row(user_id='v1nRL_f2EQs7fjh_m0TdQQ', score=0.9691872614413708, input_user_id='nOTl4aPC4tKHK35T3bNauQ'), Row(user_id='vJGLEHyhCs9V-5fAe-xx3w', score=0.9686216169669871, input_user_id='nOTl4aPC4tKHK35T3bNauQ'), Row(user_id='PGx4HvY5joEeqXzam6tO7A', score=0.9658080239271866, input_user_id='nOTl4aPC4tKHK35T3bNauQ'), Row(user_id='USUw_VT31ygQ9JkEc6nPsg', score=0.9653626156916343, input_user_id='nOTl4aPC4tKHK35T3bNauQ'), Row(user_id='yL60onlTh1BmJKVLli_K3g', score=0.9646044155730369, input_user_id='nOTl4aPC4tKHK35T3bNauQ'), Row(user_id='n4usc2DF4pd9fdi2wLDloQ', score=0.9645918982625399, input_user_id='nOTl4aPC4tKHK35T3bNauQ'), Row(user_id='ZBl

In [62]:
similar_user_rdd

ParallelCollectionRDD[499] at parallelize at PythonRDD.scala:184

In [73]:
print(similar_user_df.take(100))

[Row(user_id='xcBcIMVWEx1p7V8ACaeHpQ', score=0.9186675960618101, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'), Row(user_id='kw-YtOKPXrRB2a9wRZlmzQ', score=0.9154528761077289, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'), Row(user_id='MpN81tQOL86GaFse-_tTRQ', score=0.913263481576842, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'), Row(user_id='cNhHuEQMIpLH_qc9qGz67A', score=0.9107741561512877, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'), Row(user_id='g5W7s0n19gvT1Ujy_ITJog', score=0.9078462909984318, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'), Row(user_id='J5Eb7LhJaOa20k0ppcOCOg', score=0.9076639247312002, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'), Row(user_id='_IR48ok0ZkPMWJ2PlRCk0A', score=0.9070984934892682, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'), Row(user_id='bPUpO-bP6BmAGvSwPyDsng', score=0.9058483330903827, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'), Row(user_id='eV5usRjY2cDqNKVv8wXroA', score=0.9050774344012574, input_user_id='QBac9-Ii6jR-yLsQ5MVTHg'), Row(user_id='zkKv10FEw-iWJBHvK0mGfA', score=0.903732543